# POC for Genetic Algorithm WOLT

### Imports

In [1]:
import pygad
import numpy as np
import pandas as pd
from IPython.display import Image, display

In [2]:
from main import get_diners_constraints
# from LossFunc import user_inputs_to_loss_function_inputs, loss
from LossFunc import loss, MUST, MUST_NOT

In [3]:
PEOPLES = 3

### helper functions:

In [4]:
NO_IMAGE_URL = 'https://upload.wikimedia.org/wikipedia/commons/thumb/a/ac/No_image_available.svg/1024px-No_image_available.svg.png'

In [5]:
def print_dish(dish):
    print('-' * 50)
    print(f'{dish["name"]} - {dish["price"]} ₪')
    
    if type(dish["image"]) == str:
        display(Image(url=dish["image"], width=400, height=400))
    else:
        display(Image(url=NO_IMAGE_URL, width=200, height=200))

    print(f'vegetarian: {"✅" if dish["vegetarian"] else "❌"}', end=' ')
    print(f'GF: {"✅" if dish["GF"] else "❌"}', end=' ')
    print(f'spicy: {"✅" if dish["spicy"] else "❌"}', end=' ')
    print(f'alcohol_percentage: {"✅" if dish["alcohol_percentage"] else "❌"}')
    print('-' * 50)


In [6]:
def get_user_input(people=4, price=200, vegetarian=None, GF=None, spicy=None, alcohol_percentage=None):
    return dict(people   = people,
                price    = 80 * people if not price else price,
                vegetarian    = np.zeros(people).astype(bool) if not vegetarian    else np.array(vegetarian   ).astype(bool),
                GF   = np.zeros(people).astype(bool) if not GF   else np.array(GF  ).astype(bool),
                spicy    = np.zeros(people).astype(bool) if not spicy    else np.array(spicy   ).astype(bool),
                alcohol_percentage = np.zeros(people).astype(bool) if not alcohol_percentage else np.array(alcohol_percentage).astype(bool))

### Loss (fitness) function:

In [7]:
# def fitness_function(solution, solution_idx):
#     solution = [database.iloc[i] for i in solution]

#     total_price = sum([dish['price'] for dish in solution])
#     price_const =  total_price < user_input['price']
    
#     vegetarian_const = [dish['vegetarian'] == inp for (dish, inp) in zip(solution, user_input['vegetarian'])]    
#     GF_const = [dish['GF'] == inp for (dish, inp) in zip(solution, user_input['GF'])]    
#     spicy_const = [dish['spicy'] == inp for (dish, inp) in zip(solution, user_input['spicy'])]    
#     alcohol_percentage_const = [(dish['alcohol_percentage'] > 0) == inp for (dish, inp) in zip(solution, user_input['alcohol_percentage'])]  
    
#     hard_const = all([price_const,
#                       all(vegetarian_const),
#                       all(GF_const),
#                       all(spicy_const),
#                       all(alcohol_percentage_const)])
    
#     soft_price = (user_input['price'] - total_price) / user_input['price']
#     soft_const = np.mean([soft_price,
#                           sum(vegetarian_const)    / user_input['people'], 
#                           sum(GF_const)   / user_input['people'],
#                           sum(spicy_const)    / user_input['people'],
#                           sum(alcohol_percentage_const) / user_input['people'],
#                          ])
#     fitness = np.mean([int(hard_const), soft_const])
#     res.append(fitness)

#     return fitness

In [8]:
diner1_inputs = [True,        # - kosher
                 False,       # - vegetarian
                 False,       # - gluten free
                 False,       # - alcohol free
                 False,       # - prefer spicy
                 50,          # - max price
                 6,           # - min rating
                 0,           # - hunger level
                 ['salad'],   # - desired cuisines
                 'sunday']    # - day]

diner2_inputs = [False,       # - kosher
                 False,       # - vegetarian
                 True,        # - gluten free
                 False,       # - alcohol free
                 False,       # - prefer spicy
                 50,          # - max price
                 6,           # - min rating
                 0,           # - hunger level
                 ['salad'],   # - desired cuisines
                 'sunday']    # - day

diner3_inputs = [False,       # - kosher
                 False,       # - vegetarian
                 False,       # - gluten free
                 False,       # - alcohol free
                 True,        # - prefer spicy
                 50,          # - max price
                 6,           # - min rating
                 0,           # - hunger level
                 ['salad'],   # - desired cuisines
                 'sunday']    # - day

In [9]:
def user_inputs_to_loss_function_inputs(iner1_inputs, diner2_inputs, diner3_inputs,
                                        rest_name, meal1, meal2, meal3):
    rest_df = pd.read_csv("data/csv_wolt_restaurants_19-8-22.csv")
    meals_df = pd.read_csv("data/csv_wolt_menus_20-8-22.csv")

    rest = rest_df[rest_df["name"] == rest_name].reset_index(drop=True)
    meal1_df = meals_df[(meals_df['rest_name'] == rest_name) & (meals_df["name"] == meal1)].reset_index(drop=True)
    meal2_df = meals_df[(meals_df['rest_name'] == rest_name) & (meals_df["name"] == meal2)].reset_index(drop=True)
    meal3_df = meals_df[(meals_df['rest_name'] == rest_name) & (meals_df["name"] == meal3)].reset_index(drop=True)
    kosher1, vegetarian1, gluten_free1, alcohol_free1, spicy1, max_price1, rating1, hungry1, cuisines1, weekday = diner1_inputs
    kosher2, vegetarian2, gluten_free2, alcohol_free2, spicy2, max_price2, rating2, hungry2, cuisines2, weekday = diner2_inputs
    kosher3, vegetarian3, gluten_free3, alcohol_free3, spicy3, max_price3, rating3, hungry3, cuisines3, weekday = diner3_inputs

    # Group constraints:
    diners_kosher = False if (kosher1 == 0 and kosher2 == 0 and kosher3 == 0) else True
    diners_avg_rating = np.mean((rating1, rating2, rating3))
    hungry_diners = True if np.sum((hungry1, hungry2, hungry3)) >= 2 else False
    rest_cuisines = rest.at[0, 'food categories'].split('---')
    diner1_cui = 1 if len([meal for meal in cuisines1 if meal in rest_cuisines]) > 0 else 0
    diner2_cui = 1 if len([meal for meal in cuisines2 if meal in rest_cuisines]) > 0 else 0
    diner3_cui = 1 if len([meal for meal in cuisines3 if meal in rest_cuisines]) > 0 else 0

    O = 1 if weekday in rest.at[0, 'opening days'] else 0
    M = 1 if ((meal1_df.at[0, 'price'] + meal2_df.at[0, 'price'] + meal2_df.at[
        0, 'price']) >= 100) else 0  # TODO replace with order min
    K = 0 if diners_kosher and not rest['kosher'].values else 1
    DT = rest.at[0, 'delivery estimation [minutes]'] + rest.at[0, 'prep estimation [minutes]']
    D = 0 if (hungry_diners and rest.at[0, 'delivery estimation [minutes]'] + rest.at[
        0, 'prep estimation [minutes]'] >= HUNGRY_MINUTES) else 1
    RD = rest.at[0, 'rating'] - diners_avg_rating
    R = 1 if diners_avg_rating <= rest.at[0, 'rating'] else 0
    C = diner1_cui + diner2_cui + diner3_cui

    # individual constraints:
    diner_delivery_cost = + rest.at[0, 'delivery price'] / 3
    price1, price2, price3 = meal1_df.at[0, 'price'], meal2_df.at[0, 'price'], meal3_df.at[0, 'price']

    V1 = 0 if vegetarian1 == MUST and not meal1_df.at[0, 'vegetarian'] else 1
    V2 = 0 if vegetarian2 == MUST and not meal2_df.at[0, 'vegetarian'] else 1
    V3 = 0 if vegetarian3 == MUST and not meal3_df.at[0, 'vegetarian'] else 1
    G1 = 0 if gluten_free1 == MUST and not meal1_df.at[0, 'GF'] else 1
    G2 = 0 if gluten_free2 == MUST and not meal2_df.at[0, 'GF'] else 1
    G3 = 0 if gluten_free3 == MUST and not meal3_df.at[0, 'GF'] else 1
    A1 = 0 if alcohol_free1 == MUST and meal1_df.at[0, 'alcohol_percentage'] > 0 else 1
    A2 = 0 if alcohol_free2 == MUST and meal2_df.at[0, 'alcohol_percentage'] > 0 else 1
    A3 = 0 if alcohol_free3 == MUST and meal3_df.at[0, 'alcohol_percentage'] > 0 else 1
    spicy_meal1, spicy_meal2, spicy_meal3 = meal1_df.at[0, 'spicy'], meal2_df.at[0, 'spicy'], meal3_df.at[0, 'spicy']
    S1 = 0 if (spicy1 == MUST and not spicy_meal1) or (spicy1 == MUST_NOT and spicy_meal1) else 1
    S2 = 0 if (spicy2 == MUST and not spicy_meal2) or (spicy2 == MUST_NOT and spicy_meal2) else 1
    S3 = 0 if (spicy3 == MUST and not spicy_meal3) or (spicy3 == MUST_NOT and spicy_meal3) else 1
    PH1 = 1 if price1 + diner_delivery_cost <= max_price1 else 0
    PH2 = 1 if price2 + diner_delivery_cost <= max_price2 else 0
    PH3 = 1 if price3 + diner_delivery_cost <= max_price3 else 0
    PS1 = max_price1 - price1 if PH1 == 1 else 0
    PS2 = max_price2 - price2 if PH2 == 1 else 0
    PS3 = max_price3 - price3 if PH3 == 1 else 0
    return [O, M, K, DT, D, RD, R, C, V1, V2, V3, G1, G2, G3, A1, A2, A3, S1, S2, S3, PH1, PH2, PH3, PS1, PS2, PS3]


In [10]:
def fitness_function(solution, solution_idx):
    rest_name = database.iloc[0]['rest_name']
    meal1, meal2, meal3 = [database.iloc[i]['name'] for i in [0, 0, 0]]

    inputs = user_inputs_to_loss_function_inputs(diner1_inputs,
                                                 diner2_inputs,
                                                 diner3_inputs,
                                                 rest_name,
                                                 meal1,
                                                 meal2,
                                                 meal3
                                                )
    return -loss(*inputs)

### Initialize

In [11]:
res = []

In [12]:
# load database for a restaurant:
total_df = pd.read_csv('./data/csv_wolt_menus_20-8-22.csv')
total_df

,rest_name,name,price,alcohol_percentage,vegetarian,GF,image,days,spicy
0,Doron's Jachnun | Tel Aviv,ג׳חנון תימני עצבני,25.0,0,False,False,https://wolt-menu-images-cdn.wolt.com/menu-ima...,"[1, 2, 3, 4, 5, 6, 7]",False
1,Taqueria,‫מרגריטה ליים קפואה,30.0,150,False,False,https://wolt-menu-images-cdn.wolt.com/menu-ima...,"[1, 2, 3, 4, 5, 6, 7]",False
2,Taqueria,נאצ׳וס להכנה בתנור,45.0,0,False,False,https://wolt-menu-images-cdn.wolt.com/menu-ima...,"[1, 2, 3, 4, 5, 6, 7]",False
3,Taqueria,🌶🌶 תירס הוואנה,21.0,0,False,False,https://wolt-menu-images-cdn.wolt.com/menu-ima...,"[1, 2, 3, 4, 5, 6, 7]",True
4,Taqueria,🌶🌶🌶 כנפיים חריפות,34.0,0,False,False,https://wolt-menu-images-cdn.wolt.com/menu-ima...,"[1, 2, 3, 4, 5, 6, 7]",True
...,...,...,...,...,...,...,...,...,...
125,JARS & BOWLS,אגוז ברזיל,28.0,0,False,False,https://wolt-menu-images-cdn.wolt.com/menu-ima...,"[1, 2, 3, 4, 5, 6, 7]",False
126,JARS & BOWLS,אוכמניות ללא סוכר,42.0,0,False,False,https://wolt-menu-images-cdn.wolt.com/menu-ima...,"[1, 2, 3, 4, 5, 6, 7]",False
127,JARS & BOWLS,אננס טבעי ללא סוכר,26.0,0,False,False,https://wolt-menu-images-cdn.wolt.com/menu-ima...,"[1, 2, 3, 4, 5, 6, 7]",False
128,JARS & BOWLS,גוג׳י ברי,22.0,0,False,False,https://wolt-menu-images-cdn.wolt.com/menu-ima...,"[1, 2, 3, 4, 5, 6, 7]",False


### User input:

In [13]:
# user_input = get_user_input(people=PEOPLES, price=150,
#                             vegetarian         = [1, 0, 1, 1],
#                             GF                 = [0, 0, 0, 0],
#                             spicy              = [0, 1, 0, 0],
#                             alcohol_percentage = [0, 0, 0, 0])
# user_input

# Algorithm run:

In [14]:
best_fitness = -np.inf
best_solution = np.nan
best_res = []
best_database = np.nan

combinations = 0
for _, df in total_df.groupby('rest_name'):
    combinations += (df.shape[0] ** PEOPLES)

print(f'------- Searching space of size {combinations} -------\n')

for restaurant, database in list(total_df.groupby('rest_name'))[1:]:
    print(f' - Testing: {restaurant} combinations \n\t{database.shape[0]} possible dishes\n')
    database.reset_index(inplace=True, drop=True)
    res = []

    # choose run params:
    ga_instance = pygad.GA(num_generations         = 1000,
                           num_parents_mating      = 20,
                           sol_per_pop             = 30,
                           num_genes               = PEOPLES,

                           init_range_low          = 0,
                           init_range_high         = len(database),

                           random_mutation_min_val = 0,
                           random_mutation_max_val = len(database),

                           mutation_by_replacement = True,
                           mutation_num_genes      = 1,
                           mutation_probability    = 0.05,

                           gene_type               = int,
                           fitness_func            = fitness_function,
                          )

    # run:
    ga_instance.run()

    # print solution:
    solution, fitness, i = ga_instance.best_solution()
    
    if fitness > best_fitness:
        best_fitness = fitness
        best_res = res.copy()
        best_solution = solution
        best_database = database.copy()
    
best_solution = [best_database.iloc[i] for i in best_solution]

print(f'\n----------------- Best solution ------------------\n')
print(f'Fitness:          {best_fitness}')
print(f'Hard constraints: {"🏆" if best_fitness > 0.5 else "🛑"}')
print(f'Total price:      {sum([dish["price"] for dish in best_solution])} ₪ (limit was {user_input["price"]} ₪)')
print()
[print_dish(dish) for dish in best_solution]

print()
print('Evolution Plot:')
pd.Series(best_res).plot()
pd.Series(best_res).rolling(window=100).mean().plot()
pd.Series(np.ones(len(best_res)) * 0.5).plot(style='--')


------- Searching space of size 171976 -------

 - Testing: JARS & BOWLS combinations 
	40 possible dishes

 - Testing: Karnaf | Ibn Gvirol combinations 
	42 possible dishes

 - Testing: MeatBar Burger | Dizengoff Square combinations 
	16 possible dishes

 - Testing: Taqueria combinations 
	31 possible dishes


----------------- Best solution ------------------

Fitness:          0
Hard constraints: 🛑


NameError: name 'user_input' is not defined